In [23]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import time
from numba import njit

In [24]:
@njit
def min_vertical_seam_energy(pixel_energies):
    height, width = pixel_energies.shape
    seam_energies = np.zeros((height, width))
    back_pointers = np.zeros((height, width), dtype=np.int32)

    seam_energies[0] = pixel_energies[0]

    for y in range(1, height):
        for x in range(width):
            x_left = max(x - 1, 0)
            x_right = min(x + 1, width - 1)
            min_parent_x = x_left + np.argmin(seam_energies[y - 1, x_left:x_right + 1])

            seam_energies[y, x] = pixel_energies[y, x] + seam_energies[y - 1, min_parent_x]
            back_pointers[y, x] = min_parent_x

    min_energy = np.min(seam_energies[-1])
    return min_energy

@njit
def find_vertical_seam(pixel_energies):
    height, width = pixel_energies.shape
    seam_energies = np.zeros((height, width))
    back_pointers = np.zeros((height, width), dtype=np.int32)

    seam_energies[0] = pixel_energies[0]

    for y in range(1, height):
        for x in range(width):
            x_left = max(x - 1, 0)
            x_right = min(x + 1, width - 1)
            min_parent_x = x_left + np.argmin(seam_energies[y - 1, x_left:x_right + 1])

            seam_energies[y, x] = pixel_energies[y, x] + seam_energies[y - 1, min_parent_x]
            back_pointers[y, x] = min_parent_x

    min_seam_end_x = np.argmin(seam_energies[-1])
    seam = []

    seam_point_x = min_seam_end_x
    for y in range(height - 1, -1, -1):
        seam.append((seam_point_x, y))
        seam_point_x = back_pointers[y, seam_point_x]

    seam.reverse()
    return seam

def carve_seam(magnitude, phase, seam_path):
    height, width = magnitude.shape
    mask = np.ones((height, width), dtype=bool)
    for x, y in seam_path:
        mask[y, x] = False
    new_magnitude = magnitude[mask].reshape((height, width - 1))
    new_phase = phase[mask].reshape((height, width - 1))
    return new_magnitude, new_phase


def carve_audio(n_of_seams, magnitude, phase, is_vertical=True):
    original_height, original_width = magnitude.shape
    if is_vertical:
        for _ in range(n_of_seams):
            seam_path = find_vertical_seam(magnitude)
            magnitude, phase = carve_seam(magnitude, phase, seam_path)
    else:
        magnitude = np.rot90(magnitude, k=-1)
        phase = np.rot90(phase, k=-1)
        for _ in range(n_of_seams):
            seam_path = find_vertical_seam(magnitude)
            magnitude, phase = carve_seam(magnitude, phase, seam_path)
        magnitude = np.rot90(magnitude, k=1)
        phase = np.rot90(phase, k=1)

        new_height, new_width = magnitude.shape
        if new_height < original_height:
            pad_height = original_height - new_height
            magnitude = np.pad(magnitude, ((0, pad_height), (0, 0)), mode='constant')
            phase = np.pad(phase, ((0, pad_height), (0, 0)), mode='constant')

    return magnitude, phase

In [25]:
# Load in sounds
NJ, srNJ = librosa.load('NewJeansOMG.wav', sr=None, mono=True)
print(srNJ, "sr_flute sampling rate")
D = librosa.stft(NJ)
magnitudeNJ = np.abs(D)
phaseNJ = np.angle(D)
magnitudeNJ = np.array(magnitudeNJ)
phaseNJ = np.array(phaseNJ)
print("shape of magnitude before", magnitudeNJ.shape)
print("shape of phase before", phaseNJ.shape)
print("NJ shape (original): ", NJ.shape)

44100 sr_flute sampling rate
shape of magnitude before (1025, 1332)
shape of phase before (1025, 1332)
NJ shape (original):  (681786,)


In [26]:
n_of_seams = 800
start = time.time()
magnitude, phase = carve_audio(n_of_seams, magnitudeNJ, phaseNJ, is_vertical=True)
end = time.time()
print("shape of magnitude after", magnitude.shape)
print("shape of phase after", phase.shape)
print("The time of execution of above program is :",
      (end-start), "s")
# print("The time of execution of above program is :",
#       (end-start) * 10**3, "ms")

SystemError: CPUDispatcher(<function carve_seam at 0x12e2b0e00>) returned a result with an exception set

In [14]:
#horizontal

In [16]:
#NJ, srNJ = librosa.load('NewJeansOMG.wav', sr=None, mono=False)
NJH, srNJH = librosa.load('NewJeansOMG.wav', sr=None, mono=True)
print(srNJ, "sr_flute sampling rate")
D = librosa.stft(NJH)
magnitudeNJH = np.abs(D)
phaseNJH = np.angle(D)
magnitudeNJH = np.array(magnitudeNJH)
phaseNJH = np.array(phaseNJH)
print("shape of magnitude before", magnitudeNJH.shape)
print("shape of phase before", phaseNJH.shape)
print(srNJH)
print("D shape: ", D.shape)
print("NJH shape (original signal): ", NJH.shape)

44100 sr_flute sampling rate
shape of magnitude before (1025, 1332)
shape of phase before (1025, 1332)
44100
D shape:  (1025, 1332)
NJH shape (original signal):  (681786,)


In [18]:
# DONT RUN UNLESS YOU WANT TO DO THE CARVING
n_of_seams = 500
start = time.time()
magnitude1, phase1 = carve_audio(n_of_seams, magnitudeNJH, phaseNJH, is_vertical=False)
end = time.time()
print("shape of magnitude after", magnitude1.shape)
print("shape of phase after", phase1.shape)
print("The time of execution of above program is :",
      (end-start) * 10**3, "ms")

shape of magnitude after (1025, 1332)
shape of phase after (1025, 1332)
The time of execution of above program is : 3124.220848083496 ms
